In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import tqdm
from requests_ip_rotator import ApiGateway

from fake_useragent import UserAgent
from itertools import cycle
import random 
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import csv
import time


## Link extractor (Selenium)

In [2]:
import requests
from requests_ip_rotator import ApiGateway, EXTRA_REGIONS

In [3]:
url = "https://www.linkedin.com/jobs/search?location=Tunisia&geoId=102134353&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&currentJobId=3796559592"
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service)
driver.get(url)
time.sleep(5)





while True:
    global links
    global df1
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    links = driver.find_elements(By.CSS_SELECTOR,".base-card__full-link.absolute.top-0.right-0.bottom-0.left-0.p-0")
    l = [link.get_attribute('href') for link in links]
    df1 = pd.DataFrame(l, columns=['links'])
    df1.to_csv('linkedin_jobs_links.csv', index=False)

    
    try:
        
        more = driver.find_element(By.CSS_SELECTOR,".infinite-scroller__show-more-button.infinite-scroller__show-more-button--visible")
        time.sleep(2)
        if more:
            
            more.click()
    except:
        continue
    
    if len(links) >= 2000:
        break

    




NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.185)
Stacktrace:
	GetHandleVerifier [0x00F2C203+27395]
	(No symbol) [0x00EC3E04]
	(No symbol) [0x00DC1B7F]
	(No symbol) [0x00D9E483]
	(No symbol) [0x00E2A06F]
	(No symbol) [0x00E3C3D6]
	(No symbol) [0x00E23736]
	(No symbol) [0x00DF7541]
	(No symbol) [0x00DF80BD]
	GetHandleVerifier [0x011E3AB3+2876339]
	GetHandleVerifier [0x01237F7D+3221629]
	GetHandleVerifier [0x00FAD674+556916]
	GetHandleVerifier [0x00FB478C+585868]
	(No symbol) [0x00ECCE44]
	(No symbol) [0x00EC9858]
	(No symbol) [0x00EC99F7]
	(No symbol) [0x00EBBF4E]
	BaseThreadInitThunk [0x76CDFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771F80CE+286]
	RtlGetAppContainerNamedObjectPath [0x771F809E+238]


In [ ]:
test = pd.read_csv('links_2.csv')
test.head()

# Elements Extractor (bs4)

In [42]:
urls = pd.read_csv('linkedin_jobs_links.csv')

In [44]:
urls = urls.head(5)

In [40]:
import datetime
import re

def convert_relative_date(relative_date_str):
    """Converts a relative date string (e.g., "1 year ago", "2 months ago") to an actual date."""

    match = re.match(r"(\d+) (\w+) ago", relative_date_str)
    if not match:
        return None

    amount, unit = int(match.group(1)), match.group(2)

    now = datetime.datetime.now()

    if unit == "year" or unit == "years":
        delta = datetime.timedelta(days=amount * 365)
    elif unit == "month" or unit == "months":
        delta = datetime.timedelta(days=amount * 30)  # Approximation
    elif unit == "day" or unit == "days":
        delta = datetime.timedelta(days=amount)
    elif unit == "hour" or unit == "hours":
        delta = datetime.timedelta(hours=amount)
    elif unit == "minute" or unit == "minutes":
        delta = datetime.timedelta(minutes=amount)
    elif unit == "second" or unit == "seconds":
        delta = datetime.timedelta(seconds=amount)
    elif unit == "week" or unit == "weeks":
        delta = datetime.timedelta(weeks=amount)
    
    else:
        return None

    actual_date = now - delta
    return actual_date.strftime("%Y-%m")


In [47]:

user = UserAgent()


storage = []

def get_linkedin(urls: pd.DataFrame) -> None:

    for u in tqdm.tqdm(urls.links):
        headers =  {
            'User-Agent': user.random
        }

        url = u
        english_url = url.replace('tn.linkedin.com','www.linkedin.com') 
        
       
        try:
            request = requests.get(english_url, timeout=3600 , headers= headers)
         
        except Exception as e :
            print(e)
            continue
        
        if request.status_code != 200:
      
 
            continue
        
            
        soup = BeautifulSoup(request.text, 'html.parser')
        time.sleep(3)
        
        job_title = soup.find('h1', {'class': 'top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title'})
        job_description = soup.find('div', {'class': 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden'})
        company = soup.find('a', {'class': 'topcard__org-name-link topcard__flavor--black-link'})
        location = soup.find('span', {'class': 'topcard__flavor topcard__flavor--bullet'})
        post_date = soup.find('span', {'class': 'posted-time-ago__text topcard__flavor--metadata'})

   
        
        criteria = soup.find_all('span', {'class': 'description__job-criteria-text description__job-criteria-text--criteria'})
       
       
        
        storage.append({'post_date': convert_relative_date(post_date.text.strip()) if post_date else None,
                        'job_title':job_title.text.strip() if job_title else None, 
                        'job_description':job_description.text.strip() if job_description else None, 
                        'company':company.text.strip() if company else None,
                        'location':location.text.strip() if location else None, 
                        'seniority': criteria[0].text.strip() if criteria and len(criteria) > 0 else None,
                        'employement_type': criteria[1].text.strip() if criteria and len(criteria) > 1 else None,
                        'job_functions': criteria[2].text.strip() if criteria and len(criteria) > 2 else None,
                        'industry': criteria[3].text.strip() if criteria and len(criteria) > 3 else None,
                        
                        })
        df = pd.DataFrame(storage)
        
        df.to_csv('litest.csv',index=False)
    


    
   

In [48]:
get_linkedin(urls)


100%|██████████| 5/5 [00:23<00:00,  4.63s/it]


In [39]:
import datetime
import re

def convert_relative_date(relative_date_str):
    """Converts a relative date string (e.g., "1 year ago", "2 months ago") to an actual date."""

    match = re.match(r"(\d+) (\w+) ago", relative_date_str)
    if not match:
        return None

    amount, unit = int(match.group(1)), match.group(2)

    now = datetime.datetime.now()

    if unit == "year" or unit == "years":
        delta = datetime.timedelta(days=amount * 365)
    elif unit == "month" or unit == "months":
        delta = datetime.timedelta(days=amount * 30)  # Approximation
    elif unit == "day" or unit == "days":
        delta = datetime.timedelta(days=amount)
    elif unit == "hour" or unit == "hours":
        delta = datetime.timedelta(hours=amount)
    elif unit == "minute" or unit == "minutes":
        delta = datetime.timedelta(minutes=amount)
    elif unit == "second" or unit == "seconds":
        delta = datetime.timedelta(seconds=amount)
    elif unit == "week" or unit == "weeks":
        delta = datetime.timedelta(weeks=amount)
    
    else:
        return None

    actual_date = now - delta
    return actual_date.strftime("%Y-%m")

convert_relative_date("7 weeks ago")

'2024-06'

In [23]:
ln = pd.read_csv('linkedin_jobs_27072024.csv')

In [24]:
df = ln.copy()

In [25]:
df.post_date = df.post_date.apply(lambda x: convert_relative_date(str(x)))

In [26]:
df

,post_date,job_title,job_description,company,location,seniority,employement_type,job_functions,industry
0,2024-01,VENDEDORES/AS (m/f) SPORT ZONE BEJA,Na Sport Zone estamos empenhados em criar e ma...,Sport Zone,"Béja, Béja, Tunisia",Entry level,Full-time,Other,"Technology, Information and Internet"
1,2024-07,Réceptionniste (H/F/X),Description de l'entreprisePourquoi travailler...,Mövenpick Hotels & Resorts,"Tunis, Tunis, Tunisia",Not Applicable,Full-time,Administrative,Hospitality
2,2024-07,2D Character Artist,We are AAA Game Art Studio – art outsource stu...,AAA Game Art Studio,Tunisia,Entry level,Full-time,"Design, Art/Creative, and Information Technology",Online Audio and Video Media
3,2023-07,Oferta: ADMINISTRATIVO(A) - Beja,Oferta: ADMINISTRATIVO(A) - Beja:rhDescrição D...,RH PARTNERS DO BRASIL,"Béja, Béja, Tunisia",Entry level,Full-time,Other,Business Consulting and Services
4,2024-07,Finance Assistant (Online/Full Time),About Azurreo:Azurreo is a leading global comp...,Azurreo,Tunisia,Entry level,Full-time,Finance,Telecommunications
...,...,...,...,...,...,...,...,...,...
262,2024-07,𝐓𝐞𝐜𝐡𝐧𝐢𝐜𝐢𝐞𝐧 𝐒𝐮𝐩𝐞́𝐫𝐢𝐞𝐮𝐫 𝐒𝐮𝐩𝐩𝐨𝐫𝐭 𝐒𝐀𝐕 𝐏𝐡𝐨𝐭𝐨𝐯𝐨𝐥𝐭𝐚𝐢̈𝐪𝐮𝐞,𝐓𝐞𝐜𝐡𝐧𝐢𝐜𝐢𝐞𝐧 𝐒𝐮𝐩𝐞́𝐫𝐢𝐞𝐮𝐫 𝐒𝐮𝐩𝐩𝐨𝐫𝐭 𝐒𝐀𝐕 𝐏𝐡𝐨𝐭𝐨𝐯𝐨𝐥𝐭𝐚𝐢̈...,ENERGIKA,"Sfax, Sfax, Tunisia",Full-time,NaN,NaN,NaN
263,None,Dir-Event Services,Número de Empleo 24131599Categoría de Empleo E...,Marriott International,"Tunis, Tunis, Tunisia",Entry level,Full-time,Management and Manufacturing,Hospitality
264,2024-07,Logo Auditor,Job DescriptionJob Title: Logo AuditorReport t...,Pens.com,"Tunis, Tunisia",Entry level,Full-time,Accounting/Auditing and Finance,Advertising Services
265,2024-07,Business Intelligence Developer,"Position overview:As a BI Developer, you will ...",VEO Worldwide Services,Grand Tunis Metropolitan Area,Mid-Senior level,Full-time,Information Technology,International Trade and Development


In [27]:
df.to_csv('linkedin_jobs_27072024.csv',index=False)

In [50]:
url = "https://www.linkedin.com/jobs/search?location=Tunisia&geoId=102134353&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&currentJobId=3796559592"
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service)
driver.get(url)




df = pd.DataFrame(columns=['links'])

while True:
    global links
    global df1
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    links = driver.find_elements(By.CSS_SELECTOR,".base-card__full-link.absolute.top-0.right-0.bottom-0.left-0.p-0")
    l = [link.get_attribute('href') for link in links]
    df1 = pd.DataFrame(l, columns=['links'])
    

    
    try:
        
        more = driver.find_element(By.CSS_SELECTOR,".infinite-scroller__show-more-button.infinite-scroller__show-more-button--visible")
    
        if more:
            time.sleep(2)
            more.click()
 


        
    except:
        continue
    
    if len(df1) == len(df):
        continue
    else:
        df = pd.concat([df,df1],axis=0,ignore_index=True)

    if len(df) >= 200:
        break




In [51]:
df.shape

(300, 1)